<a href="https://colab.research.google.com/github/HungChunLi/MacroFinalReport/blob/main/TWSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step by step

## Import packages

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import warnings
import time

import pandas as pd
import numpy as np
# import cupy as cp
import matplotlib.pyplot as plt

from numpy import random
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg


In [ ]:
# # CuPy and GPU Runtime
# s = time.time()
# x_gpu = cp.ones((1000, 100, 1000))
# cp_result = cp.sqrt(cp.sum(x_gpu**2, axis=-1))
# e = time.time()
# cp_time = e - s
# print("\nTime consumed by cupy: ", cp_time)

In [ ]:
pd.set_option('display.max_rows', 99999)
pd.set_option('display.max_columns', 99999)

## Load data from Dropbox

In [ ]:
# 设置系统区域设置为 UTF-8
!apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'

# 下载文件
!wget -O Tse_06_20190102.csv "https://www.dropbox.com/scl/fi/wiepra10f739av4pwsyyl/Tse_06_20190102.csv?rlkey=gv9w5gt48l6lcertyxxtreetm&st=lknbwdzv&dl=0"

# !wget test.csv https://www.dropbox.com/scl/fi/wiepra10f739av4pwsyyl/Tse_06_20190102.csv?rlkey=gv9w5gt48l6lcertyxxtreetm&st=lknbwdzv&dl=0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
locales is already the newest version (2.35-0ubuntu3.7).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.
--2024-05-26 14:05:23--  https://www.dropbox.com/scl/fi/wiepra10f739av4pwsyyl/Tse_06_20190102.csv?rlkey=gv9w5gt48l6lcertyxxtreetm&st=lknbwdzv&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uceba0a56ec756a3a0b80ef1309e.dl.dropboxusercontent.com/cd/0/inline/CTorax10q2hkZqEUkHTKbtJZBQxwHSDeAlFCmlryjNn7ExX6LonVvFx_3kG77ZbWsQRgD5MtjPkatW4K7GbxFieFifZC2RK4tmQOEUXcbDeXpWyyJJb9jxAgXnLmpay7y53s5adUg9y4G4CMT3jSn6vb/file# [following]
--2024-05-26 14:05:24--  https://uceba0a56ec756a3a0b80ef1309e.dl.dropb

In [ ]:
PATH = '/content/Tse_06_20190102.csv'
# ?rlkey=gv9w5gt48l6lcertyxxtreetm'
TWSE = pd.read_csv(PATH, encoding='cp950')
TWSE.head(5)

<ipython-input-7-f83adf46de61>:3: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  TWSE = pd.read_csv(PATH, encoding='cp950')


,zdate,b_mkt,b_format,b_version,d_serial,stk_id7,stk_time,d_deal,d_buy_count,d_sell_count,d_five,d_deal_mark,d_buy_mark,d_sell_mark,d_instant_mark,d_trail_mark,d_trail_open_delay,d_trail_close_delay,d_trade_mode,d_open_mark,d_close_mark,vol_d,zclosed,vol_n,bp_1,bv_1,bp_2,bv_2,bp_3,bv_3,bp_4,bv_4,bp_5,bv_5,sp_1,sv_1,sp_2,sv_2,sp_3,sv_3,sp_4,sv_4,sp_5,sv_5
0,日期,市場別,格式碼,格式版本,傳輸序號,股票代號,撮合時間,成交註記,買進價量檔數,賣出價量檔數,是否揭示五檔價量,成交漲跌停註記,最佳一檔買進漲跌停註記,最佳一檔賣出漲跌停註記,瞬間價格趨勢,試算狀態註記,試算後延後開盤註記,試算後延後收盤註記,撮合方式註記,開盤註記,收盤註記,累計成交量,當盤成交價,當盤成交量,最佳一檔買進價格,最佳一檔買進數量,最佳二檔買進價格,最佳二檔買進數量,最佳三檔買進價格,最佳三檔買進數量,最佳四檔買進價格,最佳四檔買進數量,最佳五檔買進價格,最佳五檔買進數量,最佳一檔賣出價格,最佳一檔賣出數量,最佳二檔賣出價格,最佳二檔賣出數量,最佳三檔賣出價格,最佳三檔賣出數量,最佳四檔賣出價格,最佳四檔賣出數量,最佳五檔賣出價格,最佳五檔賣出數量
1,2019-01-02,01,06,03,1,00637L,2019-01-02 08:30:00.295813,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,11.42,2,11.63,10,0.00,0,0.00,0,0.00,0
2,2019-01-02,01,06,03,2,2455,2019-01-02 08:30:00.295813,0,0,1,0,0,0,2,0,1,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,71.00,15,0.00,0,0.00,0,0.00,0,0.00,0
3,2019-01-02,01,06,03,3,00672L,2019-01-02 08:30:00.400806,0,1,3,0,0,0,0,0,1,0,0,0,0,0,0,0.00,0,11.28,1,0.00,0,0.00,0,0.00,0,0.00,0,13.66,1,13.98,1,14.28,1,0.00,0,0.00,0
4,2019-01-02,01,06,03,4,3037,2019-01-02 08:30:00.400806,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,23.50,6,24.50,5,0.00,0,0.00,0,0.00,0


In [ ]:
print(TWSE.shape)
print(TWSE.columns)
print(TWSE.loc[0])

(768041, 44)
Index(['zdate', 'b_mkt', 'b_format', 'b_version', 'd_serial', 'stk_id7',
       'stk_time', 'd_deal', 'd_buy_count', 'd_sell_count', 'd_five',
       'd_deal_mark', 'd_buy_mark', 'd_sell_mark', 'd_instant_mark',
       'd_trail_mark', 'd_trail_open_delay', 'd_trail_close_delay',
       'd_trade_mode', 'd_open_mark', 'd_close_mark', 'vol_d', 'zclosed',
       'vol_n', 'bp_1', 'bv_1', 'bp_2', 'bv_2', 'bp_3', 'bv_3', 'bp_4', 'bv_4',
       'bp_5', 'bv_5', 'sp_1', 'sv_1', 'sp_2', 'sv_2', 'sp_3', 'sv_3', 'sp_4',
       'sv_4', 'sp_5', 'sv_5'],
      dtype='object')
zdate                           日期
b_mkt                          市場別
b_format                       格式碼
b_version                     格式版本
d_serial                      傳輸序號
stk_id7                       股票代號
stk_time                      撮合時間
d_deal                        成交註記
d_buy_count                 買進價量檔數
d_sell_count                賣出價量檔數
d_five                    是否揭示五檔價量
d_deal_mark                成交漲跌停註記


## Data cleaning

In [ ]:
# drop id longer than 4-digit or id below 1000 (ETF)

TWSE_trade = TWSE[1:]
print(len(TWSE_trade['stk_id7'].unique()))
TWSE_trade = TWSE_trade[TWSE_trade['stk_id7'].str.len() == 4]
print(len(TWSE_trade['stk_id7'].unique()))
TWSE_trade = TWSE_trade[TWSE_trade['stk_id7'].astype(int) > 1000]
print(len(TWSE_trade['stk_id7'].unique()))

1076
943
932


In [ ]:
TWSE_trade = TWSE_trade[TWSE_trade['d_deal'] == 1]  # deal
TWSE_trade = TWSE_trade.loc[:, ["b_mkt", "b_format", "b_version", "d_serial", "stk_id7", "stk_time", "zclosed", "vol_d"]]  # select columns
TWSE_trade.sort_values(by=['stk_id7', 'stk_time'], inplace=True)
TWSE_trade.reset_index(inplace=True, drop=True)
print(TWSE_trade.shape)
TWSE_trade.head()

(279944, 8)


,b_mkt,b_format,b_version,d_serial,stk_id7,stk_time,zclosed,vol_d
0,1,6,3,16517,1101,2019-01-02 08:35:09.524237,35.6,0
1,1,6,3,19100,1101,2019-01-02 08:37:14.878318,35.6,0
2,1,6,3,19200,1101,2019-01-02 08:37:19.893517,35.6,0
3,1,6,3,19861,1101,2019-01-02 08:37:54.992698,35.6,0
4,1,6,3,20836,1101,2019-01-02 08:38:50.30492,35.6,0


In [ ]:

# missing_count_before = TWSE_trade['zclosed'].isna().sum()
# TWSE_trade['zclosed'] = TWSE_trade.groupby('stk_id7')['zclosed'].ffill()
# missing_count_after = TWSE_trade['zclosed'].isna().sum()

# print("before:\t", missing_count_before)
# print("after:\t", missing_count_after)

In [ ]:
# test0050 = TWSE_trade[TWSE_trade['stk_id7'] == "2330"]
# plt.plot(test0050['stk_time'], test0050['vol_d'])


## Data cleaning

In [ ]:
# processing date and time
# TWSE_trade['microsecond'] = TWSE_trade['stk_time'].apply(lambda x: x[19:])
TWSE_trade['datetime'] = pd.to_datetime(TWSE_trade['stk_time'].apply(lambda x: x[:19]))
TWSE_trade['date'] = TWSE_trade['datetime'].dt.date

# calculate the time index
TWSE_trade['t'] = (TWSE_trade['datetime'].dt.hour - 9) * 60 + TWSE_trade['datetime'].dt.minute

TWSE_trade.drop(columns=['stk_time'], inplace=True)
print(TWSE_trade.shape)
TWSE_trade.head(5)

(279944, 10)


,b_mkt,b_format,b_version,d_serial,stk_id7,zclosed,vol_d,datetime,date,t
0,1,6,3,16517,1101,35.6,0,2019-01-02 08:35:09,2019-01-02,-25
1,1,6,3,19100,1101,35.6,0,2019-01-02 08:37:14,2019-01-02,-23
2,1,6,3,19200,1101,35.6,0,2019-01-02 08:37:19,2019-01-02,-23
3,1,6,3,19861,1101,35.6,0,2019-01-02 08:37:54,2019-01-02,-23
4,1,6,3,20836,1101,35.6,0,2019-01-02 08:38:50,2019-01-02,-22


In [ ]:
# transaction volume

daily_volume = TWSE_trade.sort_values('datetime').groupby("stk_id7").tail(1)
daily_volume = daily_volume.loc[:, ["datetime", "stk_id7", "vol_d"]]
daily_volume.sort_values("vol_d", inplace=True, ascending=False)
daily_volume.reset_index(drop=True, inplace=True)
print(daily_volume.shape)
daily_volume.head(5)

top_volume = list(daily_volume.loc[:,"stk_id7"])
print(len(top_volume))
print(top_volume)

(929, 3)
929
['2888', '3037', '2337', '2330', '2891', '3481', '2303', '2317', '2332', '2409', '2344', '2345', '2377', '2475', '2887', '2455', '2886', '2371', '2323', '2882', '6278', '2883', '3711', '2885', '2412', '2884', '2881', '2349', '2408', '2368', '2327', '2002', '2892', '2834', '6153', '3017', '2890', '2376', '2492', '2618', '2448', '1402', '6116', '2375', '2867', '5880', '2324', '1301', '1326', '2498', '6213', '1101', '3596', '2353', '1718', '2101', '6285', '8046', '9958', '2426', '3576', '2340', '2449', '1314', '2301', '2880', '2603', '3406', '2481', '5264', '2356', '1216', '2610', '5388', '1310', '4190', '2014', '2801', '2454', '2382', '1605', '5876', '3338', '3035', '2633', '1303', '2405', '2027', '9910', '2855', '1102', '2363', '1477', '4927', '2823', '3016', '3231', '4536', '1760', '3682', '5871', '6505', '1218', '3545', '6552', '4958', '1515', '4938', '8150', '2313', '3019', '2634', '3443', '2889', '2515', '4904', '1909', '2308', '3045', '6230', '3665', '1526', '2912', '3

In [ ]:
# select last data in each minute, each stock
TWSE_trade = TWSE_trade.loc[TWSE_trade.reset_index().groupby(['stk_id7', 'date', 't'])['index'].idxmax()]

print(TWSE_trade.shape)
TWSE_trade.head(5)

(90651, 10)


,b_mkt,b_format,b_version,d_serial,stk_id7,zclosed,vol_d,datetime,date,t
0,1,6,3,16517,1101,35.6,0,2019-01-02 08:35:09,2019-01-02,-25
3,1,6,3,19861,1101,35.6,0,2019-01-02 08:37:54,2019-01-02,-23
4,1,6,3,20836,1101,35.6,0,2019-01-02 08:38:50,2019-01-02,-22
5,1,6,3,22132,1101,35.6,0,2019-01-02 08:40:05,2019-01-02,-20
8,1,6,3,23753,1101,35.6,0,2019-01-02 08:41:40,2019-01-02,-19


## Pivot

In [ ]:
TWSE_trade_pivot = TWSE_trade.pivot_table(
  index=['stk_id7'],
  columns='t',
  values='zclosed'
).T

# fill NA by next minute or last minute
TWSE_trade_pivot = TWSE_trade_pivot.fillna(method="ffill")
TWSE_trade_pivot = TWSE_trade_pivot.fillna(method="bfill")

# select time index after 8:59 and before 13:00
TWSE_trade_pivot = TWSE_trade_pivot[(TWSE_trade_pivot.index >= 0) & (TWSE_trade_pivot.index < 240)]

# order the column by trasaction volume
TWSE_trade_pivot = TWSE_trade_pivot[top_volume]

print(TWSE_trade_pivot.shape)
TWSE_trade_pivot.head(5)

(240, 929)


stk_id7,2888,3037,2337,2330,2891,3481,2303,2317,2332,2409,2344,2345,2377,2475,2887,2455,2886,2371,2323,2882,6278,2883,3711,2885,2412,2884,2881,2349,2408,2368,2327,2002,2892,2834,6153,3017,2890,2376,2492,2618,2448,1402,6116,2375,2867,5880,2324,1301,1326,2498,6213,1101,3596,2353,1718,2101,6285,8046,9958,2426,3576,2340,2449,1314,2301,2880,2603,3406,2481,5264,2356,1216,2610,5388,1310,4190,2014,2801,2454,2382,1605,5876,3338,3035,2633,1303,2405,2027,9910,2855,1102,2363,1477,4927,2823,3016,3231,4536,1760,3682,5871,6505,1218,3545,6552,4958,1515,4938,8150,2313,3019,2634,3443,2889,2515,4904,1909,2308,3045,6230,3665,1526,2912,3673,1504,2439,2379,9904,2312,3450,6176,2347,2474,2812,4906,2456,2367,2478,1312,2201,1722,4977,6277,2485,6668,2601,1440,2348,3026,1210,2105,3034,2383,2458,3702,1802,3003,6451,2609,1532,1514,6558,2388,1710,4915,3708,8163,2520,2352,8028,2903,1215,2404,2845,6456,1316,6120,3189,3041,3321,2390,2548,1714,6269,2547,2542,8011,4989,2425,2535,1229,3698,6005,5469,2049,2534,3653,2328,3056,1434,1704,1409,2614,3706,2104,2357,9938,2838,4960,8110,3504,3059,5519,2314,3533,6239,3712,1313,3008,2329,1305,5234,4976,1904,3380,2457,3090,2354,6443,3419,2606,1609,2013,3257,4943,3661,3532,2006,2385,3356,3044,2702,2338,6670,1590,4414,3519,8039,2369,2342,2421,2444,3535,2208,9105,1786,3229,1201,2538,2395,2441,2915,8016,1701,2419,1319,2501,4438,9945,1476,3645,6674,4725,2031,6533,1733,1309,1907,2023,8112,2359,1308,9921,8261,6525,3062,4942,3036,1227,2605,6209,9907,4119,4426,2809,2841,4739,9914,6142,2704,9934,3029,6271,4919,2204,9933,6452,6202,6235,2374,2820,6282,3413,4934,1417,3049,2617,1905,1307,4306,1808,5269,4141,2505,1536,1507,8105,5288,2524,2417,2499,2109,3437,1457,4142,2107,2406,2442,6257,5608,6414,2399,2464,3686,3018,1707,4137,2705,5522,2392,8926,2415,9929,1444,2401,2546,1720,5305,3149,2530,2360,5521,2913,2207,2108,6024,3047,2816,6191,1304,5534,2607,2351,3703,1455,2615,2010,2836,2511,3454,2489,6416,4961,6196,2480,1589,2908,4766,4532,2514,1608,6449,2206,2393,5907,2022,8464,8215,1717,3005,4560,2364,2484,9911,6288,5215,2362,4994,2402,4968,2486,6431,6412,2030,1730,9935,3023,1611,2461,6283,2451,2504,1103,2316,3006,4763,1513,2103,1789,6141,2905,8482,1560,9942,3030,2465,3579,3605,3050,9925,1711,6541,1439,5259,8404,6415,1568,2509,2059,2413,2929,2239,1712,6226,1604,2358,1773,2331,2472,2355,1592,1104,9917,3042,1460,1582,1467,2637,2028,2436,3004,1447,2365,6168,2467,3033,5258,6251,2034,1109,1217,6112,2723,1471,5531,2597,1709,2069,1517,1522,5243,6155,1806,1723,3051,1432,2545,6108,2939,8101,6666,8210,1262,4540,8341,9940,3057,2497,3694,1810,2536,6215,9939,3311,2911,3704,1231,2009,2231,3011,2516,1338,2305,2707,1524,3048,1783,1459,1454,4551,2427,2527,8429,1340,1341,6214,1708,2228,3060,3312,2537,2611,9927,2102,2727,6197,1527,6177,4935,5007,1337,3058,2106,3038,2488,3376,9136,9941,3013,8033,1451,3130,2380,1463,2495,1506,1528,2420,8103,6139,2038,4746,3025,6655,4912,4104,4956,4148,1598,2020,8131,5434,3031,2612,3021,2462,4952,4108,6166,3494,2460,1437,2630,3591,3296,2302,9906,1539,1702,3052,2033,8467,4164,1464,2850,3014,3028,8213,9103,3583,1737,2015,3015,2430,2440,1809,8478,6184,3022,1472,9802,1762,2017,6591,9905,1732,9924,1413,1906,3679,2007,6477,3550,2431,1614,2636,6136,6189,3002,5607,8021,8367,6206,4557,3043,1558,1538,9908,2466,2852,6671,3024,5285,8271,2428,2613,2493,1618,5471,8422,6164,1533,2477,6192,3094,6405,1110,8114,1423,1612,3622,6531,9943,3167,1616,1731,1721,5203,3557,9902,3501,3515,3432,2706,1441,4930,2849,1519,1537,1615,2897,9926,4552,6641,4720,2851,2443,2468,4999,8411,8463,6409,6281,3054,9946,2227,8473,3705,4916,6224,1315,1108,9930,2506,2832,2062,1736,2608,3518,2731,3164,1902,1734,3530,8081,3346,1452,6243,6216,4133,1503,3055,6152,1414,2434,5525,2906,6128,3010,2024,1446,1442,2243,8249,3528,1474,4526,3040,8481,8222,4144,9919,5533,1232,6573,1449,1325,1473,1817,1521,1235,6605,1220,3536,1339,2008,8499,2387,4106,8497,3607,2642,4807,1525,2236,1468,3032,8201,2701,4764,3383,3588,1234,6289,3617,6133,3266,1735,1626,1713,5706,2029,3593,2482,8454,1203,1445,3027,59

## Modeling

### Step 2.1 Generate samples

In [ ]:
class DataSetSplitter:
    def __init__(self, data, s, t, n=30, w=3):
        self.data = data
        self.s = s
        self.t = t
        self.n = n
        self.w = w
        self.X_train_lasso, self.Y_train, self.X_test, self.Y_test, self.bmk = self.train_test_set()

    def train_test_set(self):
        data, s, t, n, w = self.data, self.s, self.t, self.n, self.w
        X_train_lasso = data.iloc[t:t+n+w-1]
        Y_train = data.iloc[t+w:t+n+w][s]
        # print("training x index: ", [t, t+n+w-1 -1])
        # print("training y index: ", [t+w, t+n+w -1])
        # print(X_train_lasso.shape, Y_train.shape)

        X_test = data.iloc[t+n:t+n+w]
        Y_test = data.iloc[t+n+w][s]
        # print("testing x index: ", [t+n, t+n+w -1])
        # print("testing y index: ", [t+n+w])
        # print(X_test.shape, Y_test.shape)

        bmk = data.iloc[t:t+n+w][s]
        # print("bmk index: ", [t, t+n+w -1])
        # print(bmk.shape)

        X_train_lasso = pd.concat([X_train_lasso, X_train_lasso.shift(-1), X_train_lasso.shift(-2)], axis=1)
        X_train_lasso = X_train_lasso.iloc[:n]
        X_train_lasso.columns = [f"{x}_{y}"  for x in data.columns for y in [1, 2, 3]]

        X_test = pd.concat([X_test, X_test.shift(-1), X_test.shift(-2)], axis=1)
        X_test = X_test.iloc[[0]]
        X_test.columns = [f"{x}_{y}"  for x in data.columns for y in [1, 2, 3]]

        return X_train_lasso, Y_train, X_test, Y_test, bmk

    def get_results(self):
        return {
            'X_train_lasso': self.X_train_lasso,
            'Y_train': self.Y_train,
            'X_test': self.X_test,
            'Y_test': self.Y_test,
            'bmk': self.bmk
        }

In [ ]:
data = TWSE_trade_pivot
s = "2330"
t = 0
n = 30
w = 3

# initialize DataSetSplitter
splitter = DataSetSplitter(data, s=s, t=t, n=30, w=3)

# training set, testing set, and benchmark
splitter_results = splitter.get_results()
X_train_lasso = splitter_results['X_train_lasso']
Y_train = splitter_results['Y_train']
X_test = splitter_results['X_test']
Y_test = splitter_results['Y_test']
bmk = splitter_results['bmk']

In [ ]:
w = 3
n = 30
s = "2330"
data = TWSE_trade_pivot

for t in range(120):
  print("minute: ", t)

  # initialize DataSetSplitter
  splitter = DataSetSplitter(data, s=s, t=t, n=30, w=3)

  # training set, testing set, and benchmark
  # skip

minute:  0
minute:  1
minute:  2
minute:  3
minute:  4
minute:  5
minute:  6
minute:  7
minute:  8
minute:  9
minute:  10
minute:  11
minute:  12
minute:  13
minute:  14
minute:  15
minute:  16
minute:  17
minute:  18
minute:  19
minute:  20
minute:  21
minute:  22
minute:  23
minute:  24
minute:  25
minute:  26
minute:  27
minute:  28
minute:  29
minute:  30
minute:  31
minute:  32
minute:  33
minute:  34
minute:  35
minute:  36
minute:  37
minute:  38
minute:  39
minute:  40
minute:  41
minute:  42
minute:  43
minute:  44
minute:  45
minute:  46
minute:  47
minute:  48
minute:  49
minute:  50
minute:  51
minute:  52
minute:  53
minute:  54
minute:  55
minute:  56
minute:  57
minute:  58
minute:  59
minute:  60
minute:  61
minute:  62
minute:  63
minute:  64
minute:  65
minute:  66
minute:  67
minute:  68
minute:  69
minute:  70
minute:  71
minute:  72
minute:  73
minute:  74
minute:  75
minute:  76
minute:  77
minute:  78
minute:  79
minute:  80
minute:  81
minute:  82
minute:  83
mi

### Step.2.2 Run Lasso

In [ ]:
class LassoModel:
    def __init__(self, data):
        self.X_train = data['X_train_lasso']
        self.Y_train = data['Y_train']
        self.X_test = data['X_test']
        self.Y_test = data['Y_test']
        self.bmk = data['bmk']
        self.lasso = LassoCV(cv=5, random_state=0, max_iter=10000)
        self.train_lasso_model()
        self.train_ar_model()

    def train_lasso_model(self):
        self.lasso.fit(self.X_train, self.Y_train)
        self.coefficient = self.lasso.coef_
        self.intercept = self.lasso.intercept_
        self.alpha = self.lasso.alpha_
        self.stock_list = self.X_test.columns[self.coefficient != 0].to_list()
        # np.where(self.coefficient != 0)[0]
        self.n_stock = len(self.stock_list)
        self.Y_predict = self.lasso.predict(self.X_test)

    def train_ar_model(self):
        self.ar_model = AutoReg(self.bmk.astype('float'), lags=3)
        self.ar_result = self.ar_model.fit()
        self.Y_predict_bmk = self.ar_result.predict(start=33, end=33).to_numpy()

    def get_results(self):
        return {
            'coefficient': self.coefficient,
            'intercept': self.intercept,
            'alpha': self.alpha,
            'stock_list': self.stock_list,
            'n_stock': self.n_stock,
            'Y_predict': self.Y_predict.item(),
            'Y_predict_bmk': self.Y_predict_bmk.item(),
            'Y_test': self.Y_test
        }

    def get_predictions(self):
        return np.array([
            self.Y_predict.item(),
            self.Y_predict_bmk.item(),
            self.Y_test
        ])

In [ ]:
# initialize the LassoModel
lasso_model = LassoModel(splitter_results)

# get result
results = lasso_model.get_results()

# print result
print('coefficient:\t', results['coefficient'])
print('intercept:\t', results['intercept'])
print('alpha:\t', results['alpha'])
print('number of stocks:\t', results['n_stock'])
print('stock list:\t', results['stock_list'])
print('Y_predict:\t', results['Y_predict'])
print('Y_predict_bmk:\t', results['Y_predict_bmk'])
print('Y_test:\t', results['Y_test'])

# get predictions
predictions = lasso_model.get_predictions()

# print predictions
print('predictions:\t', predictions)

coefficient:	 [ 0.  0.  0. ... -0. -0. -0.]
intercept:	 159.49582525845358
alpha:	 0.45792290318996504
number of stocks:	 3
stock list:	 ['2610_3', '4961_2', '4552_1']
Y_predict:	 224.44962917879718
Y_predict_bmk:	 223.48555411815684
Y_test:	 223.0
predictions:	 [224.44962918 223.48555412 223.        ]


### Step 2.3 Calculate R-square

In [ ]:
# def calculate_R_square(row):

#   x_lasso = row[:,0].reshape((-1, 1))
#   x_bmk = row[:,1].reshape((-1, 1))
#   x_lasso_standardized = StandardScaler().fit_transform(x_lasso)
#   x_bmk_standardized = StandardScaler().fit_transform(x_bmk)
#   x_combined = np.hstack((x_lasso, x_bmk))
#   y = row[:,2]

#   model1 = LinearRegression().fit(x_lasso, y)
#   model2 = LinearRegression().fit(x_bmk, y)
#   model3 = LinearRegression().fit(x_combined, y)

#   r_squared_values = np.array([
#     model1.score(x_lasso, y),
#     model1.coef_[0],
#     model1.intercept_,
#     model2.score(x_bmk, y),
#     model2.coef_[0],
#     model2.intercept_,
#     model3.score(x_combined, y),
#     model3.coef_[0],
#     model3.intercept_
#   ])

#   return r_squared_values

In [ ]:
# (n_s, n_t) = (100, 120)
# data = random.randint(-1, 1, size = (n_s, n_t, 3))

# r_squared_matrix = np.zeros((n_s, 9))
# # 計算每行的R-square
# for i in range(n_s):
#     r_squared_matrix[i] = calculate_R_square(data[i])


# print(r_squared_matrix.mean(axis=0, keepdims=False))

In [ ]:
class RSquareCalculator:
    def __init__(self, row):
        self.row = row
        self.x_lasso = row[:, 0].reshape((-1, 1))
        self.x_bmk = row[:, 1].reshape((-1, 1))
        self.y = row[:, 2]
        self.calculate_r_square()

    def calculate_r_square(self):
        # Standardize the data
        self.x_lasso_standardized = StandardScaler().fit_transform(self.x_lasso)
        self.x_bmk_standardized = StandardScaler().fit_transform(self.x_bmk)
        self.x_combined = np.hstack((self.x_lasso, self.x_bmk))

        # Train the models
        self.model1 = LinearRegression().fit(self.x_lasso, self.y)
        self.model2 = LinearRegression().fit(self.x_bmk, self.y)
        self.model3 = LinearRegression().fit(self.x_combined, self.y)

        # Create a dictionary to store the results
        self.r_squared_values = {
            'lasso_r_squared': self.model1.score(self.x_lasso, self.y),
            'lasso_coef': self.model1.coef_[0],
            'lasso_intercept': self.model1.intercept_,
            'bmk_r_squared': self.model2.score(self.x_bmk, self.y),
            'bmk_coef': self.model2.coef_[0],
            'bmk_intercept': self.model2.intercept_,
            'combined_r_squared': self.model3.score(self.x_combined, self.y)
        }

    def get_r_squared_values(self):
        return self.r_squared_values

In [ ]:
# Example usage

(n_s, n_t) = (100, 120)
data = random.randint(-1, 1, size = (n_s, n_t, 3))

# r_squared_matrix = np.zeros(n_s)
results = []
# 計算每行的R-square
for i in range(n_s):
    # r_squared_matrix[i] = calculate_R_square(data[i])
    calculator = RSquareCalculator(data[i])
    # r_squared_values = calculator.get_r_squared_values()
    # r_squared_matrix[i] = calculator.get_r_squared_values()
    results.append(calculator.get_r_squared_values())

# print(r_squared_values)
print(results)
print(len(results))


[{'lasso_r_squared': 0.006946373992775756, 'lasso_coef': -0.08335648791330928, 'lasso_intercept': -0.5423728813559322, 'bmk_r_squared': 0.022506251736593708, 'bmk_coef': 0.15004167824395662, 'bmk_intercept': -0.42372881355932207, 'combined_r_squared': 0.03125108563487944}, {'lasso_r_squared': 0.009794873090232525, 'lasso_coef': -0.09930069930069922, 'lasso_intercept': -0.5454545454545454, 'bmk_r_squared': 0.00429217906294832, 'bmk_coef': -0.06573426573426569, 'bmk_intercept': -0.5272727272727272, 'combined_r_squared': 0.0133554709620618}, {'lasso_r_squared': 0.00027972027972011926, 'lasso_coef': -0.016783216783216745, 'lasso_intercept': -0.509090909090909, 'bmk_r_squared': 0.0002874389192297766, 'bmk_coef': 0.01724633515377984, 'bmk_intercept': -0.4897959183673469, 'combined_r_squared': 0.0005985552115582715}, {'lasso_r_squared': 4.965835054715484e-06, 'lasso_coef': -0.002224694104560612, 'lasso_intercept': -0.467741935483871, 'bmk_r_squared': 0.034181121740165854, 'bmk_coef': -0.18449

### Step 2.4 Calculate mean intercept, coefficient, and R-square

In [ ]:
results_df = pd.DataFrame(results)
results_df.head()

,lasso_r_squared,lasso_coef,lasso_intercept,bmk_r_squared,bmk_coef,bmk_intercept,combined_r_squared
0,0.006946,-0.083356,-0.542373,0.022506,0.150042,-0.423729,0.031251
1,0.009795,-0.099301,-0.545455,0.004292,-0.065734,-0.527273,0.013355
2,0.000280,-0.016783,-0.509091,0.000287,0.017246,-0.489796,0.000599
3,0.000005,-0.002225,-0.467742,0.034181,-0.184496,-0.557377,0.034316
4,0.003023,0.055413,-0.434783,0.027515,0.165324,-0.377049,0.029478


# One shot

In [ ]:
n_s = 10  # number of stocks
n_t = 12
data = TWSE_trade_pivot
n = 30  # sample
w = 3  # window

# r_squared_matrix_1 = np.zeros((n_s, 9))
results = []
stock_list_matrix = np.empty((10, 12), dtype=object)
for i, s in enumerate(top_volume[:n_s]):

    r_squared_matrix_2 = np.zeros((n_t, 3))
    for j, t in enumerate(range(60, 60+n_t), start=0):
        print(i, "stock: ", s, "\t", j, "minute: ", t)

        # initialize DataSetSplitter
        splitter = DataSetSplitter(data, s=s, t=t, n=30, w=3)
        splitter_results = splitter.get_results()

        # initialize the LassoModel
        lasso_model = LassoModel(splitter_results)
        model_predictions = lasso_model.get_predictions()
        stock_list = lasso_model.get_results()['stock_list']

        r_squared_matrix_2[j] = model_predictions
        stock_list_matrix[i][j] = stock_list

    # R-square for each stock
    results.append(r_squared_matrix_2)
    # r_squared_matrix_1[i] = calculate_R_square(r_squared_matrix_2)

0 stock:  2888 	 0 minute:  60


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


0 stock:  2888 	 1 minute:  61


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


0 stock:  2888 	 2 minute:  62


...

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


9 stock:  2409 	 10 minute:  70


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


9 stock:  2409 	 11 minute:  71


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
# result_df = pd.DataFrame(results)
print(results[0].shape)

(12, 3)


In [ ]:
result_df.to_csv("result2_1perc.csv")

In [ ]:
result_df.shape

In [ ]:
np.mean(result_df[0])

In [ ]:
np.mean(result_df[3])

In [ ]:
np.mean(result_df[6])

In [ ]:
np.mean(result_df[1])

In [ ]:
a = result_df[0].to_numpy()
print(a)
print(type(a))

In [ ]:
np.std(a, ddof=1) / np.sqrt(np.size(a))

In [ ]:
np.mean(result_df[2])

In [ ]:
a = result_df[2].to_numpy()
print(a)
print(type(a))

In [ ]:
np.std(a, ddof=1) / np.sqrt(np.size(a))

In [ ]:
stock_list_matrix[3]

array([list(['2881_1', '2610_3', '1210_2', '3437_1', '3023_3', '2905_3', '2472_1', '3021_3', '4552_1', '2832_1', '1452_2', '8480_2']),
       list(['2610_3']), list(['2610_3']),
       list(['2881_1', '2368_2', '1402_3', '2610_3', '4938_1', '1526_1', '3026_2', '3257_3', '9921_3', '1905_3', '4141_2', '3437_1', '1455_2', '3023_3', '2905_3', '6415_2', '2472_1', '3004_2', '2102_2', '3021_3', '8422_2', '6192_2', '1612_3', '4552_1', '1452_2', '4106_3']),
       list(['2344_1', '2610_3', '4938_1', '3026_2', '1210_2', '3257_3', '3437_1', '2392_3', '1455_2', '4961_2', '2905_3', '2472_1', '5471_2', '6192_2']),
       list(['2610_3', '4961_2', '4552_1']), list(['4961_2', '4552_1']),
       list(['4961_2', '4552_1']), list(['4961_2', '4552_1']),
       list(['4961_2', '4552_1']), list(['4552_1']), list(['4552_1'])],
      dtype=object)

In [ ]:
stock_list_matrix.shape

(10, 12)

In [ ]:
data_list = [list(row) for row in stock_list_matrix[3]]

data_1d = [item for sublist in data_list for item in sublist]

In [ ]:
data_1d

['2881_1',
 '2610_3',
 '1210_2',
 '3437_1',
 '3023_3',
 '2905_3',
 '2472_1',
 '3021_3',
 '4552_1',
 '2832_1',
 '1452_2',
 '8480_2',
 '2610_3',
 '2610_3',
 '2881_1',
 '2368_2',
 '1402_3',
 '2610_3',
 '4938_1',
 '1526_1',
 '3026_2',
 '3257_3',
 '9921_3',
 '1905_3',
 '4141_2',
 '3437_1',
 '1455_2',
 '3023_3',
 '2905_3',
 '6415_2',
 '2472_1',
 '3004_2',
 '2102_2',
 '3021_3',
 '8422_2',
 '6192_2',
 '1612_3',
 '4552_1',
 '1452_2',
 '4106_3',
 '2344_1',
 '2610_3',
 '4938_1',
 '3026_2',
 '1210_2',
 '3257_3',
 '3437_1',
 '2392_3',
 '1455_2',
 '4961_2',
 '2905_3',
 '2472_1',
 '5471_2',
 '6192_2',
 '2610_3',
 '4961_2',
 '4552_1',
 '4961_2',
 '4552_1',
 '4961_2',
 '4552_1',
 '4961_2',
 '4552_1',
 '4961_2',
 '4552_1',
 '4552_1',
 '4552_1']

In [ ]:
from collections import Counter

counts = Counter(data_1d)
counts

Counter({'2881_1': 2,
         '2610_3': 6,
         '1210_2': 2,
         '3437_1': 3,
         '3023_3': 2,
         '2905_3': 3,
         '2472_1': 3,
         '3021_3': 2,
         '4552_1': 9,
         '2832_1': 1,
         '1452_2': 2,
         '8480_2': 1,
         '2368_2': 1,
         '1402_3': 1,
         '4938_1': 2,
         '1526_1': 1,
         '3026_2': 2,
         '3257_3': 2,
         '9921_3': 1,
         '1905_3': 1,
         '4141_2': 1,
         '1455_2': 2,
         '6415_2': 1,
         '3004_2': 1,
         '2102_2': 1,
         '8422_2': 1,
         '6192_2': 2,
         '1612_3': 1,
         '4106_3': 1,
         '2344_1': 1,
         '2392_3': 1,
         '4961_2': 6,
         '5471_2': 1})

In [ ]:
sorted(counts)

['1210_2',
 '1402_3',
 '1452_2',
 '1455_2',
 '1526_1',
 '1612_3',
 '1905_3',
 '2102_2',
 '2344_1',
 '2368_2',
 '2392_3',
 '2472_1',
 '2610_3',
 '2832_1',
 '2881_1',
 '2905_3',
 '3004_2',
 '3021_3',
 '3023_3',
 '3026_2',
 '3257_3',
 '3437_1',
 '4106_3',
 '4141_2',
 '4552_1',
 '4938_1',
 '4961_2',
 '5471_2',
 '6192_2',
 '6415_2',
 '8422_2',
 '8480_2',
 '9921_3']

In [ ]:
# w = 3
# n = 30
# s = "0050"

# Y_test_array = np.array([])
# Y_predict_lasso_array = np.array([])
# Y_predict_bmk_array = np.array([])

# for t in range(0, 5):
#   print("minute: ", t)

#   X_train_lasso = TWSE_trade_pivot.iloc[t:t+n+w-1]
#   Y_train = TWSE_trade_pivot.iloc[t+w:t+n+w][s]
#   # print("training x index: ", [t, t+n+w-1 -1])
#   # print("training y index: ", [t+w, t+n+w -1])

#   X_test = TWSE_trade_pivot.iloc[t+n:t+n+w]
#   Y_test = TWSE_trade_pivot.iloc[t+n+w][s]
#   # print("testing x index: ", [t+n, t+n+w -1])
#   # print("testing y index: ", [t+n+w])

#   X_train_bmk = TWSE_trade_pivot.iloc[t:t+n+w][s]
#   # print("bmk index: ", [t, t+n+w -1])

#   X_train_lasso = pd.concat([X_train_lasso, X_train_lasso.shift(-1), X_train_lasso.shift(-2)], axis=1)
#   X_train_lasso = X_train_lasso.iloc[:n]
#   # print(X_train_lasso.shape)

#   X_test = pd.concat([X_test, X_test.shift(-1), X_test.shift(-2)], axis=1)
#   X_test = X_test.iloc[0]
#   X_test = X_test.to_numpy().reshape(1, -1)
#   # print(X_test.shape)

#   lasso = LassoCV(cv=5, random_state=0, max_iter=10000)
#   lasso.fit(X_train_lasso, Y_train)

#   # coefficient = lasso.coef_
#   # intercept = lasso.intercept_
#   # alpha = lasso.alpha_
#   # print('coefficient\t', coefficient)
#   # print('intercept\t', intercept)
#   # print('alpha\t', alpha)
#   Y_predict_lasso = lasso.predict(X_test)

#   X_train_bmk.astype('float')
#   train_bmk = X_train_bmk[:w+n-1]
#   test_bmk = X_train_bmk[w+n-1]
#   model = AutoReg(train_bmk, lags=w)
#   result = model.fit()
#   Y_predict_bmk = result.predict(start=w+n, end=w+n).to_numpy()

#   Y_test_array = np.append(Y_test_array, float(Y_test))
#   Y_predict_lasso_array = np.append(Y_predict_lasso_array, float(Y_predict_lasso))
#   Y_predict_bmk_array = np.append(Y_predict_bmk_array, float(Y_predict_bmk))

#   print(Y_test, Y_predict_lasso, Y_predict_bmk)


In [ ]:
# r_square_1, r_square_2, r_square_3 = calculate_R_square(Y_test_array.reshape(-1, 1), Y_predict_lasso_array.reshape(-1, 1), Y_predict_bmk_array.reshape(-1, 1))
# print(r_square_1, r_square_2, r_square_3)
